In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product as product
from scipy.linalg import expm
%matplotlib notebook

In [2]:
N = 100
I_0 = 5
Rsi = 2/N
Rir = 1/N

states = {(x, y) : ind for ind, (x, y) in enumerate([(x, y) for x in range(N + 1) for y in range(N + 1) if x + y <= N], 0)}
biStates = {ind : state for state, ind in states.items()}

In [3]:
Q = np.zeros((len(states), len(states)))

for (x, y), ind in states.items():
    if y > 0:
        transitionTo = states[(x, y - 1)]
        Q[ind, transitionTo] = Rir * y
    if x > 0:
        transitionTo = states[(x - 1, y + 1)]
        Q[ind, transitionTo] = Rsi * y * x
    Q[ind, ind] = - np.sum(Q[ind, :])

In [28]:
def doobGillespie(diagonal, currState, Pjump, biStates):
    times = [(0, biStates[currState])]
    state = currState
    while biStates[state][1] != 0: # While the number of infectives isn't zero
        timeSpent = np.random.exponential(scale=1/(diagonal[state]))
        state = np.random.choice(range(Pjump.shape[1]), p=Pjump[state, :])
        times.append((times[-1][0] + timeSpent, biStates[state]))
    return times

In [29]:
diagonal =  - np.diagonal(Q)
Pjump = Q / diagonal[:, np.newaxis]
np.fill_diagonal(Pjump, 0)
np.nan_to_num(Pjump, copy=False)

C:\Users\Aviral\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00502513, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [32]:
check = doobGillespie(diagonal, states[(N - I_0, I_0)], Pjump, biStates)

In [31]:
def binSearch(searchIn, goal):
    if goal >= searchIn[-1][0]:
        return (len(searchIn) - 1, None)
    elif goal <= searchIn[0][0]:
        return (None, 0)
    else:
        low = 0
        upper = len(searchIn) - 1
        middle = int((upper + low)/2)
        while low < upper: # replace this
            # print("searching between " + str(low) + " and " + str(upper))
            if middle < len(searchIn) - 1:
                if searchIn[middle][0] <= goal and searchIn[middle + 1][0] >= goal:
                    return (middle, middle + 1)
            elif middle > 0:
                if searchIn[middle - 1][0] >= goal and searchIn[middle][0] <= goal:
                    return (middle, middle - 1)
            
            if goal > searchIn[middle][0]:
                low = middle + 1
            else: # goal < middle
                upper = middle - 1
            middle = int((upper + low)/2)
        return (middle, middle + 1)

In [38]:
binSearch(check, 5)
print(check[97])
print(check[98])

(3.904218061626356, (0, 98))
(6.439502610125743, (0, 97))


In [39]:
ts = [5, 10, 20, 50]

t_values = np.asarray([0 for t in ts])
trials = 1000

diagonal = - np.diag(Q)

for trial in range(trials):
    trace = doobGillespie(diagonal, states[(N - I_0, I_0)], Pjump, biStates)
    for ind_t, t in enumerate(ts, 0):
        lower, _ = binSearch(trace, t)
        t_values[ind_t] += trace[lower][1][1]

In [40]:
[val/trials for val in t_values]

[96.41, 91.892, 83.158, 61.466]